### Agent logging with lifecycle hooks

This notebook demonstrates using OpenAI's Agent lifecycle hooks to print the steps the agent is taking to solve the task.

Based on the [agent lifecycle example](https://github.com/openai/openai-agents-python/blob/main/examples/basic/agent_lifecycle_example.py) in the openai agents examples.

This gives us some idea of what is happening without having to use the traces screen.

It also demonstrates using a class to instantiate multiple MCP servers from multiple configs, which makes using several MCP servers at once more ergonomic in a notebook.

In [14]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents_logging import AgentLoggingHooks
from mcp_server_manager import MCPServers
import os

In [ ]:
load_dotenv(override=True)

In [ ]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
brave_api_key = os.getenv("BRAVE_API_KEY")

server_configs = {
    'files': {'params': {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}},
    'browser': {'params': {"command": "npx", "args": ["@playwright/mcp@latest"]}},
    'brave': {'params': {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": {"BRAVE_API_KEY": brave_api_key}}}
}


instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
You search the web to find good information, and then use the web URLs to navigate to the content you need.
You can save files to the sandbox, and you can read files from the sandbox.
If you save the user's request to the sandbox, you say so, rather than replying with the file content or a summary of the file content.
"""

input = """
I want a good recipe for a 500g loaf of sourdough bread.
Use a search and find the best recipe from the results.
Then use your browser automation tools navigate to that recipe via its URL.
From the page full content, give a detailed summary of the recipe and save it to sandbox/recipe.md in markdown format.
If a page doesn't load, try another one.
"""


async with MCPServers(server_configs) as mcp:
    agent = Agent(
        name="Researcher", 
        instructions=instructions, 
        model="gpt-4o-mini",
        mcp_servers=mcp.get_all_servers(),
        hooks=AgentLoggingHooks(display_name="Researcher")
        )
    with trace("investigate"):
        result = await Runner.run(agent, input)
        print("\n\n=== FINAL OUTPUT ===\n\n")
        print(result.final_output)


### Check out the trace

https://platform.openai.com/traces